In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import Levenshtein
import numpy as np
import math
import statistics


In [2]:
df = pd.read_csv("/Users/nsjain/Downloads/Keystrokes/files/5_keystrokes.txt", delimiter="\t")
metadata_df = pd.read_csv("/Users/nsjain/Downloads/Sample/metadata_participants.txt", delimiter="\t")

In [3]:
df

,PARTICIPANT_ID,TEST_SECTION_ID,SENTENCE,USER_INPUT,KEYSTROKE_ID,PRESS_TIME,RELEASE_TIME,LETTER,KEYCODE
0,5,7,The others raise their eyebrows.,The others raise their eyebrows,204,1471934383592,1471934383760,SHIFT,16
1,5,7,The others raise their eyebrows.,The others raise their eyebrows,203,1471934383701,1471934383760,T,84
2,5,7,The others raise their eyebrows.,The others raise their eyebrows,205,1471934383838,1471934383910,h,72
3,5,7,The others raise their eyebrows.,The others raise their eyebrows,206,1471934383910,1471934383991,e,69
4,5,7,The others raise their eyebrows.,The others raise their eyebrows,207,1471934384054,1471934384138,,32
...,...,...,...,...,...,...,...,...,...
625,5,83,If lunch doesn't work then how about coffee?,Id lunch doesn't work then how about coffee?,3565,1471934546993,1471934547098,f,70
626,5,83,If lunch doesn't work then how about coffee?,Id lunch doesn't work then how about coffee?,3574,1471934547207,1471934547267,e,69
627,5,83,If lunch doesn't work then how about coffee?,Id lunch doesn't work then how about coffee?,3575,1471934547361,1471934547433,e,69
628,5,83,If lunch doesn't work then how about coffee?,Id lunch doesn't work then how about coffee?,3576,1471934547495,1471934547647,SHIFT,16


In [4]:
df['LETTER'] = df['LETTER'].replace(['SHIFT', 'BKSP', 'DEL'], '')

LATENCIES

In [5]:
df['PRESS_TIME'] = pd.to_numeric(df['PRESS_TIME'])
df['RELEASE_TIME'] = pd.to_numeric(df['RELEASE_TIME'])

df['PL'] = df['PRESS_TIME'].diff()
df['HL'] = df['RELEASE_TIME'] - df['PRESS_TIME']
df['RL'] = df['RELEASE_TIME'].diff()

df.head()


,PARTICIPANT_ID,TEST_SECTION_ID,SENTENCE,USER_INPUT,KEYSTROKE_ID,PRESS_TIME,RELEASE_TIME,LETTER,KEYCODE,PL,HL,RL
0,5,7,The others raise their eyebrows.,The others raise their eyebrows,204,1471934383592,1471934383760,,16,NaN,168,NaN
1,5,7,The others raise their eyebrows.,The others raise their eyebrows,203,1471934383701,1471934383760,T,84,109.0,59,0.0
2,5,7,The others raise their eyebrows.,The others raise their eyebrows,205,1471934383838,1471934383910,h,72,137.0,72,150.0
3,5,7,The others raise their eyebrows.,The others raise their eyebrows,206,1471934383910,1471934383991,e,69,72.0,81,81.0
4,5,7,The others raise their eyebrows.,The others raise their eyebrows,207,1471934384054,1471934384138,,32,144.0,84,147.0


DIGRAPHS

In [6]:
selected_columns = ['PARTICIPANT_ID', 'TEST_SECTION_ID', 'LETTER', 'PRESS_TIME', 'RELEASE_TIME']
kdf = df.loc[:, selected_columns].copy()
kdf.head()

,PARTICIPANT_ID,TEST_SECTION_ID,LETTER,PRESS_TIME,RELEASE_TIME
0,5,7,,1471934383592,1471934383760
1,5,7,T,1471934383701,1471934383760
2,5,7,h,1471934383838,1471934383910
3,5,7,e,1471934383910,1471934383991
4,5,7,,1471934384054,1471934384138


In [7]:
kdf['Digraphs'] = kdf['LETTER'] + kdf['LETTER'].shift(-1)

kdf['HL'] = kdf['RELEASE_TIME'] - kdf['PRESS_TIME'].shift(-1)
kdf['IL'] = kdf['PRESS_TIME'].shift(-1) - kdf['RELEASE_TIME'].shift(-1)
kdf['PL'] = kdf['PRESS_TIME'] - kdf['PRESS_TIME'].shift(-1)
kdf['RL'] = kdf['RELEASE_TIME'] - kdf['RELEASE_TIME'].shift(-1)

kdf = kdf.drop(['LETTER'], axis=1)

kdf.head()

,PARTICIPANT_ID,TEST_SECTION_ID,PRESS_TIME,RELEASE_TIME,Digraphs,HL,IL,PL,RL
0,5,7,1471934383592,1471934383760,T,59.0,-59.0,-109.0,0.0
1,5,7,1471934383701,1471934383760,Th,-78.0,-72.0,-137.0,-150.0
2,5,7,1471934383838,1471934383910,he,0.0,-81.0,-72.0,-81.0
3,5,7,1471934383910,1471934383991,e,-63.0,-84.0,-144.0,-147.0
4,5,7,1471934384054,1471934384138,o,-100.0,-118.0,-184.0,-218.0


In [8]:
kdf = kdf[kdf['Digraphs'].str.strip().str.len() == 2]
kdf['HL'] = kdf['HL'].apply(lambda x: x**2)
kdf['IL'] = kdf['IL'].apply(lambda x: x**2)
kdf['RL'] = kdf['RL'].apply(lambda x: x**2)
kdf['PL'] = kdf['PL'].apply(lambda x: x**2)
kdf

,PARTICIPANT_ID,TEST_SECTION_ID,PRESS_TIME,RELEASE_TIME,Digraphs,HL,IL,PL,RL
1,5,7,1471934383701,1471934383760,Th,6084.0,5184.0,18769.0,22500.0
2,5,7,1471934383838,1471934383910,he,0.0,6561.0,5184.0,6561.0
5,5,7,1471934384238,1471934384356,ot,9216.0,14400.0,45796.0,46656.0
6,5,7,1471934384452,1471934384572,th,3969.0,10816.0,33489.0,27889.0
7,5,7,1471934384635,1471934384739,he,961.0,12544.0,5329.0,6561.0
...,...,...,...,...,...,...,...,...,...
622,5,83,1471934546528,1471934546630,co,2116.0,15625.0,3136.0,6241.0
623,5,83,1471934546584,1471934546709,of,11881.0,6561.0,54756.0,36100.0
624,5,83,1471934546818,1471934546899,ff,8836.0,11025.0,30625.0,39601.0
625,5,83,1471934546993,1471934547098,fe,11881.0,3600.0,45796.0,28561.0


In [9]:
kdf.describe()

,PARTICIPANT_ID,TEST_SECTION_ID,PRESS_TIME,RELEASE_TIME,HL,IL,PL,RL
count,369.0,369.000000,3.690000e+02,3.690000e+02,369.000000,369.000000,369.000000,369.000000
mean,5.0,42.867209,1.471934e+12,1.471934e+12,9838.268293,10242.425474,26611.563686,27465.924119
std,0.0,23.491817,4.789030e+04,4.788948e+04,38081.241237,5335.343799,51964.291311,53585.560116
min,5.0,7.000000,1.471934e+12,1.471934e+12,0.000000,1936.000000,2116.000000,0.000000
25%,5.0,25.000000,1.471934e+12,1.471934e+12,256.000000,6084.000000,9025.000000,7056.000000
50%,5.0,43.000000,1.471934e+12,1.471934e+12,2025.000000,10000.000000,14400.000000,17161.000000
75%,5.0,60.000000,1.471935e+12,1.471935e+12,4489.000000,13456.000000,22801.000000,27556.000000
max,5.0,83.000000,1.471935e+12,1.471935e+12,518400.000000,32041.000000,651249.000000,636804.000000


In [10]:
def compute_series_record(values):
    count = len(values)
    features = {"count": count}

    if count > 0:
        features["mean"] = statistics.mean(values)
        features["median"] = statistics.median(values)
        features["pstdev"] = statistics.pstdev(values)
        features["max"] = max(values)
        features["min"] = min(values)
    else:
        features["mean"] = None
        features["median"] = None
        features["pstdev"] = None
        features["max"] = None
        features["min"] = None

    return features

In [11]:
def assign_series(root, mount, values):
    root[mount] = compute_series_record(values)

In [12]:
def calculate_digraphs(df):
    character_bigram_raw_il = {}
    character_bigram_raw_pl = {}
    character_bigram_raw_rl = {}
    character_bigram_raw_hl = {}
    character_bigram_log_il = {}
    character_bigram_log_pl = {}
    character_bigram_log_rl = {}
    character_bigram_log_hl = {}
    character_bigram_counts = {}

    for _, row in df.iterrows():
        digraph = row["Digraphs"]
        il = row["IL"]
        hl = row["HL"]
        pl = row["PL"]
        rl = row["RL"]

        if isinstance(digraph, str) and len(digraph) <= 2:
            log_il = math.log(il) if il is not None and il > 0 else 0
            log_hl = math.log(hl) if hl is not None and hl > 0 else 0
            log_pl = math.log(pl) if pl is not None and pl > 0 else 0
            log_rl = math.log(rl) if rl is not None and rl > 0 else 0

            index = digraph

            if index in character_bigram_counts:
                raw_il_list = character_bigram_raw_il[index]
                raw_pl_list = character_bigram_raw_pl[index]
                raw_rl_list = character_bigram_raw_rl[index]
                raw_hl_list = character_bigram_raw_hl[index]
                log_il_list = character_bigram_log_il[index]
                log_pl_list = character_bigram_log_pl[index]
                log_rl_list = character_bigram_log_rl[index]
                log_hl_list = character_bigram_log_hl[index]
                character_bigram_counts[index] += 1
            else:
                raw_il_list = []
                raw_pl_list = []
                raw_rl_list = []
                raw_hl_list = []
                log_il_list = []
                log_pl_list = []
                log_rl_list = []
                log_hl_list = []

                character_bigram_counts[index] = 1
                character_bigram_raw_il[index] = raw_il_list
                character_bigram_raw_pl[index] = raw_pl_list
                character_bigram_raw_rl[index] = raw_rl_list
                character_bigram_raw_hl[index] = raw_hl_list
                character_bigram_log_il[index] = log_il_list
                character_bigram_log_pl[index] = log_pl_list
                character_bigram_log_rl[index] = log_rl_list
                character_bigram_log_hl[index] = log_hl_list

            raw_il_list.append(il)
            raw_pl_list.append(pl)
            raw_rl_list.append(rl)
            raw_hl_list.append(hl)

            log_il_list.append(log_il)
            log_pl_list.append(log_pl)
            log_rl_list.append(log_rl)
            log_hl_list.append(log_hl)

    digraphs = {}
    for key in character_bigram_counts:
        assign_series(digraphs, key + "_raw_il", character_bigram_raw_il[key])
        assign_series(digraphs, key + "_raw_pl", character_bigram_raw_pl[key])
        assign_series(digraphs, key + "_raw_rl", character_bigram_raw_rl[key])
        assign_series(digraphs, key + "_raw_hl", character_bigram_raw_hl[key])
        assign_series(digraphs, key + "_log_il", character_bigram_log_il[key])
        assign_series(digraphs, key + "_log_pl", character_bigram_log_pl[key])
        assign_series(digraphs, key + "_log_rl", character_bigram_log_rl[key])
        assign_series(digraphs, key + "_log_hl", character_bigram_log_hl[key])

    return digraphs

In [13]:
digraph_features = calculate_digraphs(kdf)
print(digraph_features)

{'Th_raw_il': {'count': 5, 'mean': 8259.6, 'median': 7744.0, 'pstdev': 3789.016579536173, 'max': 14161.0, 'min': 3600.0}, 'Th_raw_pl': {'count': 5, 'mean': 25842.8, 'median': 23104.0, 'pstdev': 7705.976003077092, 'max': 40804.0, 'min': 18769.0}, 'Th_raw_rl': {'count': 5, 'mean': 35724.2, 'median': 38416.0, 'pstdev': 7670.632109546123, 'max': 44944.0, 'min': 22500.0}, 'Th_raw_hl': {'count': 5, 'mean': 10326.2, 'median': 8649.0, 'pstdev': 4742.1224530794225, 'max': 19600.0, 'min': 6084.0}, 'Th_log_il': {'count': 5, 'mean': 8.90487999082301, 'median': 8.954673628956414, 'pstdev': 0.48965296369245087, 'max': 9.55824698622306, 'min': 8.1886891244442}, 'Th_log_pl': {'count': 5, 'mean': 10.122372582908241, 'median': 10.047761041692553, 'pstdev': 0.2619689878564904, 'max': 10.61653539480241, 'min': 9.83996185165625}, 'Th_log_rl': {'count': 5, 'mean': 10.45686025032506, 'median': 10.556229318461034, 'pstdev': 0.2401788610857195, 'max': 10.713172549344025, 'min': 10.021270588192511}, 'Th_log_hl'

In [14]:
def implode(obj, tgt=None, namespace=""):
    if tgt is None:
        tgt = {}

    for key in obj:
        val = obj[key]

        if namespace == "":
            ref = key
        else:
            ref = namespace + "_" + key

        if type(val) is dict:
            implode(val, tgt, ref)
        else:
            tgt[ref] = val

    return tgt

In [15]:
final_set = implode(digraph_features)
print(len(final_set))
print(final_set)

8064
{'Th_raw_il_count': 5, 'Th_raw_il_mean': 8259.6, 'Th_raw_il_median': 7744.0, 'Th_raw_il_pstdev': 3789.016579536173, 'Th_raw_il_max': 14161.0, 'Th_raw_il_min': 3600.0, 'Th_raw_pl_count': 5, 'Th_raw_pl_mean': 25842.8, 'Th_raw_pl_median': 23104.0, 'Th_raw_pl_pstdev': 7705.976003077092, 'Th_raw_pl_max': 40804.0, 'Th_raw_pl_min': 18769.0, 'Th_raw_rl_count': 5, 'Th_raw_rl_mean': 35724.2, 'Th_raw_rl_median': 38416.0, 'Th_raw_rl_pstdev': 7670.632109546123, 'Th_raw_rl_max': 44944.0, 'Th_raw_rl_min': 22500.0, 'Th_raw_hl_count': 5, 'Th_raw_hl_mean': 10326.2, 'Th_raw_hl_median': 8649.0, 'Th_raw_hl_pstdev': 4742.1224530794225, 'Th_raw_hl_max': 19600.0, 'Th_raw_hl_min': 6084.0, 'Th_log_il_count': 5, 'Th_log_il_mean': 8.90487999082301, 'Th_log_il_median': 8.954673628956414, 'Th_log_il_pstdev': 0.48965296369245087, 'Th_log_il_max': 9.55824698622306, 'Th_log_il_min': 8.1886891244442, 'Th_log_pl_count': 5, 'Th_log_pl_mean': 10.122372582908241, 'Th_log_pl_median': 10.047761041692553, 'Th_log_pl_pstd

In [17]:
import os
from tqdm import tqdm

def create_feature_set(directory):
    final_set = {}

    file_list = os.listdir(directory)

    with tqdm(total=len(file_list), desc="Processing files") as progress_bar:
        for filename in os.listdir(directory):
            if not filename.endswith("_keystrokes.txt"):
                continue

            user_id = filename.split("_")[0]
            filepath = os.path.join(directory, filename)

            try:
                df = pd.read_csv(filepath, delimiter="\t", encoding='latin-1')
                df['LETTER'] = df['LETTER'].replace(['SHIFT', 'BKSP', 'DEL', 'CAPS_LOCK'], '')
                df['PRESS_TIME'] = pd.to_numeric(df['PRESS_TIME'], errors='coerce')
                df['RELEASE_TIME'] = pd.to_numeric(df['RELEASE_TIME'], errors='coerce')

                df['PL'] = df['PRESS_TIME'].diff()
                df['HL'] = df['RELEASE_TIME'] - df['PRESS_TIME']
                df['RL'] = df['RELEASE_TIME'].diff()

                selected_columns = ['PARTICIPANT_ID', 'TEST_SECTION_ID', 'LETTER', 'PRESS_TIME', 'RELEASE_TIME']
                kdf = df.loc[:, selected_columns].copy()
                kdf['Digraphs'] = kdf['LETTER'] + kdf['LETTER'].shift(-1)

                kdf['HL'] = kdf['RELEASE_TIME'] - kdf['PRESS_TIME'].shift(-1)
                kdf['IL'] = kdf['PRESS_TIME'].shift(-1) - kdf['RELEASE_TIME'].shift(-1)
                kdf['PL'] = kdf['PRESS_TIME'] - kdf['PRESS_TIME'].shift(-1)
                kdf['RL'] = kdf['RELEASE_TIME'] - kdf['RELEASE_TIME'].shift(-1)

                kdf = kdf.drop(['LETTER'], axis=1)
                kdf['Digraphs'] = kdf['Digraphs'].astype(str)
                kdf = kdf[kdf['Digraphs'].str.strip().str.len() == 2]
                kdf['HL'] = kdf['HL'].apply(lambda x: x**2)
                kdf['IL'] = kdf['IL'].apply(lambda x: x**2)
                kdf['RL'] = kdf['RL'].apply(lambda x: x**2)
                kdf['PL'] = kdf['PL'].apply(lambda x: x**2)

                digraph_features = calculate_digraphs(kdf)
                user_features = implode(digraph_features)
                final_set[user_id] = user_features
                
                progress_bar.update(1)

            except FileNotFoundError:
                continue
            
            except pd.errors.ParserError:
                continue

        return final_set

directory = "/Users/nsjain/Downloads/Keystrokes/files"

feature_sets = create_feature_set(directory)

Processing files:  51%|█████     | 85527/168595 [43:28<51:15, 27.01it/s]  

: 

: 

In [ ]:
import csv

def store_feature_set(feature_set, filename):
    features = set()
    for user_features in feature_set.values():
        features.update(user_features.keys())

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['user_id'] + sorted(features))
        writer.writeheader()
        for user_id, user_features in feature_set.items():
            row = {'user_id': user_id}
            row.update(user_features)
            writer.writerow(row)

filename = 'final.csv'

store_feature_set(feature_sets, filename)
